In [1]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [2]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [3]:
# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [4]:
train[0][1]

0

In [5]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [6]:
# Define the model
W_sm = model.add_lookup_parameters((nwords, ntags)) # Word weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [7]:
# A function to calculate scores for one value
def calc_scores(words):
    dy.renew_cg()
    score = dy.esum([dy.lookup(W_sm, x) for x in words])
    b_sm_exp = dy.parameter(b_sm)
    return score + b_sm_exp

In [9]:
for ITER in range(100):
    # Perform training
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    for words, tag in train:
        my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
        train_loss += my_loss.value()
        my_loss.backward()
        trainer.update()
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
    # Perform testing
    test_correct = 0.0
    for words, tag in dev:
        scores = calc_scores(words).npvalue()
        predict = np.argmax(scores)
        if predict == tag:
            test_correct += 1
    print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=2.4147, time=0.42s
iter 0: test acc=0.2425
iter 1: train loss/sent=2.0441, time=0.39s
iter 1: test acc=0.2593
iter 2: train loss/sent=1.8603, time=0.39s
iter 2: test acc=0.2692
iter 3: train loss/sent=1.7264, time=0.38s
iter 3: test acc=0.2833
iter 4: train loss/sent=1.6187, time=0.39s
iter 4: test acc=0.2878
iter 5: train loss/sent=1.5300, time=0.38s
iter 5: test acc=0.2928
iter 6: train loss/sent=1.4525, time=0.39s
iter 6: test acc=0.3018
iter 7: train loss/sent=1.3847, time=0.38s
iter 7: test acc=0.3050
iter 8: train loss/sent=1.3233, time=0.41s
iter 8: test acc=0.3136
iter 9: train loss/sent=1.2693, time=0.39s
iter 9: test acc=0.3149
iter 10: train loss/sent=1.2197, time=0.39s
iter 10: test acc=0.3195
iter 11: train loss/sent=1.1742, time=0.60s
iter 11: test acc=0.3226
iter 12: train loss/sent=1.1335, time=0.39s
iter 12: test acc=0.3208
iter 13: train loss/sent=1.0956, time=0.40s
iter 13: test acc=0.3186
iter 14: train loss/sent=1.0605, time=0.39s
iter 14: t